In [1]:
import os
import codecs

In [2]:
# 定义地址
bert_model_path = "/Users/zhouwencheng/Desktop/Grass/data/model/ImportModel/BERT/chinese_L-12_H-768_A-12"
config_path = os.path.join(bert_model_path, 'bert_config.json')
checkpoint_path = os.path.join(bert_model_path, 'bert_model.ckpt')
vocab_path = os.path.join(bert_model_path, 'vocab.txt')

In [3]:
# 读取 vocab 文件  加载为字典

os.environ['TF_KERAS'] = '1'
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_inv = {v: k for k, v in token_dict.items()}
print(len(token_dict)) 

21128


In [5]:
# 载入模型
from keras_bert import load_trained_model_from_checkpoint

In [6]:
model = load_trained_model_from_checkpoint(config_path, checkpoint_path,  training=True)
model.summary(line_length=120)

W0811 12:55:49.346699 4321588096 deprecation.py:506] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0811 12:55:50.194427 4321588096 deprecation.py:506] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             [(None, 512)]              0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (21128, 16226304      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment

## Predict Masked

In [7]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)
text = '数学是利用符号语言研究数量、结构、变化以及空间等概念的一门学科'
tokens = tokenizer.tokenize(text)
tokens[1] = tokens[2] = '[MASK]'
print('Tokens:', tokens)

Tokens: ['[CLS]', '[MASK]', '[MASK]', '是', '利', '用', '符', '号', '语', '言', '研', '究', '数', '量', '、', '结', '构', '、', '变', '化', '以', '及', '空', '间', '等', '概', '念', '的', '一', '门', '学', '科', '[SEP]']


In [8]:
import numpy as np
indices = np.array([[token_dict[token] for token in tokens] + [0] * (512 - len(tokens))])
segments = np.array([[0] * len(tokens) + [0] * (512 - len(tokens))])
masks = np.array([[0, 1, 1] + [0] * (512 - 3)])

In [9]:
print("indices:", indices[0][:10])
print("segments:", segments[0][:10])
print("masks:", masks[0][:10])

indices: [ 101  103  103 3221 1164 4500 5016 1384 6427 6241]
segments: [0 0 0 0 0 0 0 0 0 0]
masks: [0 1 1 0 0 0 0 0 0 0]


In [13]:
predicts = model.predict([indices, segments, masks])
# [0].argmax(axis=-1).tolist()
# print('Fill with: ', list(map(lambda x: token_dict_inv[x], predicts[0][1:3])))

In [29]:
type(predicts[1])
predicts
# predicts[1][0]
predicts[0].shape

(1, 512, 21128)

## Predict Next Sentence

In [27]:
sentence_1 = '数学是利用符号语言研究數量、结构、变化以及空间等概念的一門学科。'
sentence_2 = '从某种角度看屬於形式科學的一種。'
print('Tokens:', tokenizer.tokenize(first=sentence_1, second=sentence_2))
indices, segments = tokenizer.encode(first=sentence_1, second=sentence_2, max_len=512)
masks = np.array([[0] * 512])

Tokens: ['[CLS]', '数', '学', '是', '利', '用', '符', '号', '语', '言', '研', '究', '數', '量', '、', '结', '构', '、', '变', '化', '以', '及', '空', '间', '等', '概', '念', '的', '一', '門', '学', '科', '。', '[SEP]', '从', '某', '种', '角', '度', '看', '屬', '於', '形', '式', '科', '學', '的', '一', '種', '。', '[SEP]']


In [31]:
predicts = model.predict([np.array([indices]), np.array([segments]), masks])[1]
print('%s is random next: ' % sentence_2, bool(np.argmax(predicts, axis=-1)[0]))

从某种角度看屬於形式科學的一種。 is random next:  False


In [34]:
sentence_2 = '任何一个希尔伯特空间都有一族标准正交基。'
print('Tokens:', tokenizer.tokenize(first=sentence_1, second=sentence_2))
indices, segments = tokenizer.encode(first=sentence_1, second=sentence_2, max_len=512)

Tokens: ['[CLS]', '数', '学', '是', '利', '用', '符', '号', '语', '言', '研', '究', '數', '量', '、', '结', '构', '、', '变', '化', '以', '及', '空', '间', '等', '概', '念', '的', '一', '門', '学', '科', '。', '[SEP]', '任', '何', '一', '个', '希', '尔', '伯', '特', '空', '间', '都', '有', '一', '族', '标', '准', '正', '交', '基', '。', '[SEP]']


In [35]:
predicts = model.predict([np.array([indices]), np.array([segments]), masks])[1]
print('%s is random next: ' % sentence_2, bool(np.argmax(predicts, axis=-1)[0]))

任何一个希尔伯特空间都有一族标准正交基。 is random next:  True
